In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Weather as uw
import Utilities.SnD as us
import APIs.QuickStats as qs

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [41]:
uw.open_w_sel_file()

In [117]:
#region declarations
corn_states=['IA','IL','IN','OH','MO','MN','SD','NE']
years=range(1985,2023)

# Select the Weather Stations
df_w_sel = uw.get_w_sel_df()
df_w_sel = df_w_sel[df_w_sel[uw.WS_COUNTRY_ALPHA] == 'USA']
# df_w_sel = df_w_sel[df_w_sel[uw.WS_STATE_ALPHA] == 'IL']

# Build the Weather DF
w_vars = [uw.WV_PREC,  uw.WV_TEMP_MAX]
in_files = uw.WS_UNIT_ALPHA
out_cols = uw.WS_UNIT_ALPHA
w_df_all = uw.build_w_df_all(df_w_sel, w_vars, in_files, out_cols)

# # Build the Weights
# weights = us.get_USA_prod_weights('CORN', 'STATE', years, corn_states)

# # Weighted DataFrame
# w_w_df_all = uw.weighted_w_df_all(w_df_all, weights, output_column='USA')

# uw.add_Sdd(w_df_all['hist'])

w_df_all['hist']
test=w_df_all['hist']
test=test.reset_index()
test.loc[0,'IA_TempMax']=3
test.iloc[0]['IA_TempMax']

3.0

In [106]:
uw.seasonalize(w_df_all['hist'][['IL_Prec']])

year,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2018,2019,2020,2021,2022,Max,Min,Mean,2022_Proj,2021_Analog
seas_day,,,,,,,,,,,,,,,,,,,,,
2020-01-01,24.751257,0.000000,0.518822,0.081835,1.203202,0.137657,0.001691,0.010543,1.189390,0.229380,...,5.204113e-03,19.125038,0.000559,6.509508,5.913129,24.751257,0.000000,2.353056,5.913129,6.509508
2020-01-02,0.364159,0.372394,2.534792,0.000002,0.038434,0.021513,0.000024,3.127471,0.113829,0.182303,...,1.580972e-07,0.568643,0.130972,9.088188,11.429700,18.169506,0.000000,1.655474,11.429700,9.088188
2020-01-03,0.123576,1.625257,0.077099,0.004867,0.144736,0.636168,0.128143,2.639375,4.031518,1.927928,...,3.864557e-02,0.014615,1.710339,2.203819,0.006433,15.416859,0.000000,1.936101,0.006433,2.203819
2020-01-04,2.048156,0.106056,0.047969,0.004857,0.032557,6.541734,0.238444,0.023150,28.091349,0.414854,...,1.986835e-02,0.584286,1.076784,0.007596,0.000000,28.091349,0.000000,2.628746,0.000000,0.007596
2020-01-05,0.218555,0.471986,0.258947,0.000643,1.815001,0.300522,6.906959,0.000841,16.291571,0.159165,...,4.699804e-02,2.987932,0.034798,0.038710,0.008074,22.043743,0.000000,2.695555,0.008074,0.038710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,0.354525,0.000000,4.606472,12.649376,1.244174,0.843803,0.000000,0.049512,0.087108,0.000284,...,6.958124e+00,0.000600,0.008420,3.392663,NaN,43.151485,0.000000,2.786531,2.786531,3.392663
2020-12-28,0.092417,0.010723,18.208427,16.559114,0.096699,8.472308,0.236394,0.185424,0.436430,0.000284,...,9.068862e+00,1.805190,0.178888,1.595287,NaN,24.022327,0.000000,3.146206,3.146206,1.595287
2020-12-29,0.149477,0.097182,2.475560,0.149242,1.178247,19.424675,1.177753,8.449707,0.232650,0.000235,...,1.969858e-01,21.558014,0.426382,13.338152,NaN,32.250935,0.000235,3.349162,3.349162,13.338152


In [67]:
df_to_ext =  w_df_all[uw.WD_H_GFS]
w_w_df_ext = uw.extend_with_seasonal_df(df_to_ext, modes=[uw.EXT_MEAN])

w_w_df_ext

,IA_Prec,IA_TempMax,IL_Prec,IL_TempMax,IN_Prec,IN_TempMax,MN_Prec,MN_TempMax,MO_Prec,MO_TempMax,NE_Prec,NE_TempMax,OH_Prec,OH_TempMax,SD_Prec,SD_TempMax
time,,,,,,,,,,,,,,,,
1985-01-01,6.397065,-7.441345,24.751257,5.421037,24.211063,12.146098,0.367761,-15.641856,29.888802,4.210868,0.066882,-8.846363,7.685587,15.672124,0.009242,-13.866185
1985-01-02,0.021137,-10.093124,0.364159,-0.863381,1.984503,8.158212,0.021630,-15.554237,0.468136,-4.238242,0.000000,-5.467136,3.086113,16.564726,0.003087,-8.298045
1985-01-03,0.000000,-7.362988,0.123576,-7.325004,0.128540,-2.879271,0.001129,-8.955413,0.038166,-4.003171,0.000000,0.804736,0.038566,0.356918,0.000000,-1.043913
1985-01-04,0.000000,-3.441880,2.048156,-4.484976,2.942260,-0.969538,0.000307,-1.195709,2.440511,-2.588277,0.000000,5.469008,0.975676,-0.000238,0.000000,3.232912
1985-01-05,0.005071,2.741907,0.218555,0.114381,0.561939,0.470925,0.041556,1.699730,0.133436,2.410379,0.000000,7.879215,1.303611,1.636065,0.009307,4.515441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,1.218934,-0.083769,2.786531,3.155150,2.768928,3.025820,1.173553,-4.721293,3.848792,6.234104,0.903475,3.413456,2.052352,2.522306,0.591900,-0.400118
2022-12-28,1.026092,-0.278738,3.146206,3.617415,3.011825,3.915145,0.854115,-4.982265,3.338981,6.101626,0.578369,2.928559,2.235706,3.944188,0.565948,-0.911432
2022-12-29,1.810501,0.488334,3.349162,3.597818,3.129383,3.938471,1.218357,-5.205576,2.230508,6.234203,0.636872,2.988279,2.229464,4.033361,0.606714,-1.686513


In [101]:
def return_multi():
    return 1,2

In [103]:
test1, _ = return_multi()

In [104]:
test1

1

In [84]:
test1, test2 = return_multi()

In [22]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
        
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')

Files:
SD_TempAvg_gfs.csv (15CjUUaTF472P1BQEUNJO_nhjtP_Ac9eg)
SD_TempMin_gfs.csv (14WYBt8JUXgTGpsczpiCTHlumUnSWithy)
SD_TempMax_gfs.csv (10AIb9jeltMrBDk0KOMPeyrpt5q5Zm5Up)
SD_Prec_gfs.csv (1202k-tHWue4xACcgsigklEjkm3ndaQbT)
OH_TempAvg_gfs.csv (13bYHWIhaIWeqMzq57qx4DOO3bp4VSoij)
OH_TempMin_gfs.csv (116q7VJT5GrXy1s3Bil_PTEtOHZEvzBcl)
OH_TempMax_gfs.csv (1-Dr-lK0jHDiQWlKXuwKPJ3I36CaxebL6)
OH_Prec_gfs.csv (11gON3bke-UMJNi-0hBfbeQKtRhTcVadB)
NE_TempAvg_gfs.csv (13dB922LFFDUQdUZGWpgnvPqjnU9SQIPk)
NE_TempMin_gfs.csv (122O1IMp4fiFosbrK_kd2giBMq-9OucwJ)


In [ ]:
import io

In [24]:
import pandas as pd
import Utilities.Weather as uw

In [26]:
test=uw.get_w_sel_df()

In [36]:
cols=list(test.columns)
cols.remove('country_name')
cols

['amuIds',
 'idBlock',
 'country_alpha',
 'country_code',
 'unit_name',
 'unit_alpha',
 'unit_code',
 'state_name',
 'state_alpha',
 'state_code',
 'Prec',
 'TempAvg',
 'TempMin',
 'TempMax',
 'TempSurf',
 'Soil',
 'Humi',
 'VVI']

In [ ]:
file_id = '1-7mnO-RX9GiCWsLqF5ngo0a4Z5o4je3W'
path = 'https://drive.google.com/uc?export=download&id='+file_id
df = pd.read_csv(path)
df

In [ ]:
# https://stackoverflow.com/questions/52135293/google-drive-api-the-user-has-not-granted-the-app-error

In [ ]:
# https://discuss.streamlit.io/t/google-drive-csv-file-link-to-pandas-dataframe/8057

In [ ]:
# https://developers.google.com/drive/api/v2/reference/files/get

In [ ]:
from apiclient import errors
from apiclient import http
# ...

def print_file_metadata(service, file_id):
  """Print a file's metadata.

  Args:
    service: Drive API service instance.
    file_id: ID of the file to print metadata for.
  """

  return(service.files().get(fileId=file_id).execute())


def print_file_content(service, file_id):
  """Print a file's content.

  Args:
    service: Drive API service instance.
    file_id: ID of the file.

  Returns:
    File's content if successful, None otherwise.
  """
  # print (service.files().get_media(fileId=file_id).execute())
  return service.files().get_media(fileId=file_id).execute()


def download_file(service, file_id, local_fd):
  """Download a Drive file's content to the local filesystem.

  Args:
    service: Drive API Service instance.
    file_id: ID of the Drive file that will downloaded.
    local_fd: io.Base or file object, the stream that the Drive file's
        contents will be written to.
  """
  request = service.files().get_media(fileId=file_id)
  media_request = http.MediaIoBaseDownload(local_fd, request)

  while True:
    download_progress, done = media_request.next_chunk()

    if download_progress:
      print ('Download Progress: %d%%' % int(download_progress.progress() * 100))
    if done:
      print ('Download Complete')
      return

In [ ]:
file_id='1dWZkKoknSvOS2PKkxGmIyh5HtYYd1HZ3'

test=pd.read_csv(print_file_content(service,file_id))

# f = open("test.csv", "r", encoding="utf-8")

# download_file(service,file_id, test)
test

In [3]:
import Utilities.Weather as uw

In [17]:
sel_states=['IL','IA']
sel_df=uw.get_w_sel_df()

if True:
    test=sel_df[sel_df['state_alpha'].isin(sel_states)]
else:
    sel_df=sel_df.set_index('state_alpha',drop=False)
    test=sel_df.loc[sel_states]


test


,amuIds,idBlock,country_name,country_alpha,country_code,unit_name,unit_alpha,unit_code,state_name,state_alpha,state_code,Prec,TempAvg,TempMin,TempMax,TempSurf,Soil,Humi,VVI
0,1018108,129,United States,USA,840,Iowa,IA,wip,Iowa,IA,wip,y,NaN,y,y,NaN,NaN,NaN,NaN
1,1018106,129,United States,USA,840,Illinois,IL,wip,Illinois,IL,wip,y,NaN,y,y,NaN,NaN,NaN,NaN
